<a href="https://colab.research.google.com/github/RogerHeederer/NLP_entry/blob/master/ELMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference Source : wikidocs.net 유영준님 자료

스스로 학습하면서 필요한 부분에는 추가적 설명, 소스 코드 삽입 및 수정 등이 있습니다. 영리적 목적이 아닌, 자기 계발 목적으로 정리한 자료입니다.

##엘모 개념 나름 정리##

2018년에 제안된 새로운 임베딩 방법론

Word Sense(중의성)을 반영하지 못하는 워드투백, 글로브의 단점을 극복하고자 새로운 방법론을 제안

Bi-directional Language Model을 활용. Forward , backward LM 둘다를 사용해서 임베딩 벡터의 품질을 개선한다. 

I want to give you the present, new computer

Do your best at present.  같은 present 단어이지만, 문맥에 따라 선물, 현재로 다르게 해석될 수 있고, 벡터값을 다르게 표현해준다 

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
pip install tensorflow-hub

In [3]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [4]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin-1')
data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
data['v1'] = data['v1'].replace(['ham', 'spam'], [0,1])
y_data = list(data['v1'])
x_data = list(data['v2'])

In [7]:
x_data[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [8]:
print(y_data[:5])

[0, 0, 1, 0, 0]


In [9]:
print(len(x_data))

5572


In [10]:
# 8:2로 훈련 데이터와 트레인 데이터 분리
n_of_train = int(len(x_data) * 0.8)
n_of_test = int(len(x_data) - n_of_train)
print(n_of_train)
print(n_of_test)

4457
1115


In [11]:
x_train = np.asarray(x_data[:n_of_train])
x_test = np.asarray(x_data[n_of_train:])
y_train = np.asarray(y_data[:n_of_train])
y_test = np.asarray(y_data[n_of_train:])

In [13]:
def ELMoEmbedding(x):
  return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]
#데이터의 이동이 케라스 -> 텐서플로우 -> 케라스가 되도록 하는 함수

In [14]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype = tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024,))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)

model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
history = model.fit(x_train, y_train, epochs=1, batch_size=60)

Epoch 1/1
4457/4457 [==============================] - 1099s 246ms/step - loss: 0.1305 - accuracy: 0.9513
